In [ ]:
# S06 – Task 7 (rumore ambientale): analisi e logbook
# Autori: Alessia Di Nino, Marco Malucchi (T10) – Tecnologie Digitali, UniPi A.A. 2025/2026

import numpy as np, pathlib, textwrap, json
import matplotlib.pyplot as plt

# ---------- Utilità ----------
def load_txt(path):
    """Legge file con 3 colonne: time [s], ch1 [V], ch2 [V], saltando eventuali righe che iniziano per '#'."""
    raw = []
    with open(path, "r", encoding="utf-8") as f:
        for ln in f:
            s = ln.strip()
            if not s or s.startswith("#"):  # commenti del demoscope / header
                continue
            parts = s.replace(",",".").split()
            if len(parts) >= 3:
                raw.append([float(parts[0]), float(parts[1]), float(parts[2])])
    A = np.array(raw, float)
    t, ch1, ch2 = A[:,0], A[:,1], A[:,2]
    return t, ch1, ch2

def robust_stats(x):
    """RMS (std) attorno alla media, picco-picco e IQR utile come metrica robusta."""
    mu = np.mean(x)
    rms = np.std(x, ddof=1)
    p2p = np.ptp(x)
    iq = np.subtract(*np.percentile(x, [75, 25]))
    return mu, rms, p2p, iq

def window_minmax(x, m=20):
    """Min/max a finestra scorrevole di m campioni (per ‘alone’)."""
    # pad per comodità
    k = m//2
    xpad = np.pad(x, (k,k), mode="edge")
    # rolling min/max veloci via reshape in blocchetti
    # (per semplicità qui usiamo un approccio non super-ottimizzato ma chiaro)
    out_min, out_max = np.empty_like(x), np.empty_like(x)
    for i in range(len(x)):
        seg = xpad[i:i+m]
        out_min[i] = seg.min()
        out_max[i] = seg.max()
    return out_min, out_max

def simple_psd(x, fs):
    """PSD monolato con finestra di Hann e media su blocchi (tipo Welch semplificato)."""
    n = len(x)
    if n < 256:
        nfft = 256
    else:
        nfft = 1024
    step = nfft//2
    win = np.hanning(nfft)
    k = max(1, (n - nfft) // step + 1)
    acc = 0.0
    for i in range(k):
        s = i*step
        seg = x[s:s+nfft]
        if len(seg) < nfft: break
        X = np.fft.rfft((seg - np.mean(seg))*win)
        P = (np.abs(X)**2) / (np.sum(win**2) * fs)
        acc = acc + P
    Pxx = acc / max(1, i+1)
    f = np.fft.rfftfreq(nfft, 1/fs)
    return f, Pxx

def dominant_freq(f, Pxx, fmin=40, fmax=200):
    """Picco di potenza in [fmin,fmax] Hz (utile per rete a 50/100 Hz, PWM luci, ecc.)."""
    m = (f>=fmin) & (f<=fmax)
    if not np.any(m):
        return None, None
    j = np.argmax(Pxx[m])
    fj = f[m][j]
    pj = Pxx[m][j]
    return float(fj), float(pj)

def analyze_one(label, t, ch1, ch2, outdir="out"):
    out = {}
    pathlib.Path(outdir).mkdir(exist_ok=True, parents=True)
    dt = np.median(np.diff(t))
    fs = 1.0/dt

    # Statistiche canali (offset ~ media; rumore ~ RMS)
    stats = {}
    for name, x in (("Ch1", ch1), ("Ch2", ch2)):
        mu, rms, p2p, iq = robust_stats(x)
        stats[name] = dict(mean=mu, rms=rms, p2p=p2p, iqr=iq)

    # ‘Alone’ semitrasparente (min/max a finestra)
    mwin = max(10, int(0.002*fs))  # ~2 ms di finestra se possibile
    mn1, mx1 = window_minmax(ch1, mwin)
    mn2, mx2 = window_minmax(ch2, mwin)

    # PSD e frequenza dominante
    f1, P1 = simple_psd(ch1 - np.mean(ch1), fs)
    f2, P2 = simple_psd(ch2 - np.mean(ch2), fs)
    fpeak1, ppeak1 = dominant_freq(f1, P1)
    fpeak2, ppeak2 = dominant_freq(f2, P2)

    # Grafico temporale con ‘alone’
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(t*1e3, ch1, lw=1, label="Ch1")
    ax.plot(t*1e3, ch2, lw=1, label="Ch2")
    ax.fill_between(t*1e3, mn1, mx1, alpha=0.25, color="tab:orange")
    ax.fill_between(t*1e3, mn2, mx2, alpha=0.25, color="tab:blue")
    ax.set_xlabel("Time [msec]"); ax.set_ylabel("Signal [V]"); ax.grid(True, alpha=.3)
    ax.legend()
    fig.tight_layout(); fig.savefig(f"{outdir}/{label}_time.png", dpi=140); plt.close(fig)

    # Grafico PSD
    fig, ax = plt.subplots(figsize=(10,4))
    ax.semilogy(f1, P1, label="Ch1")
    ax.semilogy(f2, P2, label="Ch2")
    ax.set_xlim(0, min(500, f2[-1] if len(f2) else 500))
    ax.set_xlabel("Frequency [Hz]"); ax.set_ylabel("PSD [V^2/Hz]"); ax.grid(True, alpha=.3)
    if fpeak1: ax.axvline(fpeak1, ls="--", lw=1, alpha=.5, color="tab:orange")
    if fpeak2: ax.axvline(fpeak2, ls="--", lw=1, alpha=.5, color="tab:blue")
    ax.legend()
    fig.tight_layout(); fig.savefig(f"{outdir}/{label}_psd.png", dpi=140); plt.close(fig)

    # Risultati
    out["fs"] = fs
    out["stats"] = stats
    out["fpeak"] = dict(Ch1=fpeak1, Ch2=fpeak2)
    out["files"] = dict(time=f"{outdir}/{label}_time.png", psd=f"{outdir}/{label}_psd.png")
    return out

# ---------- ESECUZIONE ----------
files = {
    "stanza": "stanza.txt",
    "coperto": "coperto.txt",
    "flash": "flash.txt",
    "lampada_treno": "lampada_treno.txt",
}

results = {}
for key, path in files.items():
    t, ch1, ch2 = load_txt(path)
    results[key] = analyze_one(key, t, ch1, ch2, outdir="out")

# ---------- Logbook Markdown autogenerato ----------
def fmt(v, unit="V", digits=4):
    return f"{v:.{digits}g} {unit}"

def block_for(key, R):
    fs = R["fs"]
    ch1 = R["stats"]["Ch1"]; ch2 = R["stats"]["Ch2"]
    fpk1, fpk2 = R["fpeak"]["Ch1"], R["fpeak"]["Ch2"]
    txt = f"""
### {key}

- **Frequenza di campionamento**: {fs:.1f} Hz  
- **Ch1** (ad es. ingresso LED o riferimento):  
  - offset = {fmt(ch1['mean'])}, noise RMS = {fmt(ch1['rms'])}, p-p = {fmt(ch1['p2p'])}
- **Ch2** (uscita amplificatore / fotodiodo):  
  - offset = {fmt(ch2['mean'])}, noise RMS = {fmt(ch2['rms'])}, p-p = {fmt(ch2['p2p'])}
- **Frequenza dominante** (banda 40–200 Hz): Ch1 = {fpk1 if fpk1 else '—'} Hz, Ch2 = {fpk2 if fpk2 else '—'} Hz

![Time](./{R['files']['time']})
![PSD](./{R['files']['psd']})

**Commento tecnico.** Il segnale del fotodiodo dovrebbe essere *quasi* continuo in condizioni stazionarie,
ma compaiono oscillazioni per tre motivi tipici: (i) **sfarfallio della rete**: luci alimentate a 50 Hz producono una
modulazione a **100 Hz** (raddrizzamento) chiaramente visibile nella PSD; (ii) **driver/PWM** di LED o torce del telefono,
che introducono armoniche nella banda 100–1 kHz; (iii) **rumore dell’elettronica** (op-amp, quantizzazione ADC, rumore di shot).
La **banda semitrasparente blu/arancio** è costruita come *min–max a finestra scorrevole* e rappresenta **l’inviluppo**
delle fluttuazioni su brevi intervalli: se si allarga, significa che nel breve periodo la variabilità (rumore + ripple)
aumenta anche a offset quasi costante.
"""
    return textwrap.dedent(txt).strip()

md = """# S06 — Task 7: Stima del rumore ambientale

Questo report riassume i risultati ottenuti misurando il segnale a fotodiodo con diverse condizioni di illuminazione
(*stanza*, *coperto*, *flash*, *lampada treno*). Per ciascun caso si riportano offset, rumore RMS e picco-picco
dei due canali, oltre alla frequenza dominante nella banda 40–200 Hz (utile per identificare rete, PWM, ecc.).

---

"""

for key in ["stanza","coperto","flash","lampada_treno"]:
    md += block_for(key, results[key]) + "\n\n---\n\n"

# Append nota metodologica e riferimenti
md += textwrap.dedent("""
## Nota metodologica
- L’**alone** è calcolato come min/max su finestre di ~2 ms (aggiustate automaticamente in base a fs).
- La **PSD** è stimata con una FFT a blocchi con finestra di Hann e media (stile Welch semplificato).
- Il **rumore RMS** è la deviazione standard attorno alla media (stima non-bias, ddof=1).

""")

# Scrive file
pathlib.Path("out").mkdir(exist_ok=True)
with open("out/S06_Task7_logbook.md", "w", encoding="utf-8") as f:
    f.write(md)

print("✅ Analisi completata. File e grafici in ./out/   Apri: out/S06_Task7_logbook.md")
print(json.dumps({k:results[k]["stats"] for k in results}, indent=2))


✅ Analisi completata. File e grafici in ./out/   Apri: out/S06_Task7_logbook.md
{
  "stanza": {
    "Ch1": {
      "mean": 0.0055398639521889274,
      "rms": 0.0006785850741038907,
      "p2p": 0.0036826663655330095,
      "iqr": 0.0
    },
    "Ch2": {
      "mean": 0.22757291006534824,
      "rms": 0.04037369586155667,
      "p2p": 0.12304714831850105,
      "iqr": 0.07084532781974301
    }
  },
  "coperto": {
    "Ch1": {
      "mean": 0.005759241538416968,
      "rms": 0.001091165409292101,
      "p2p": 0.007365332731066019,
      "iqr": 0.0
    },
    "Ch2": {
      "mean": 0.009611374670910609,
      "rms": 0.0009598900869760643,
      "p2p": 0.003728701464197001,
      "iqr": 0.0
    }
  },
  "flash": {
    "Ch1": {
      "mean": 0.005575827490914835,
      "rms": 0.000763172870250752,
      "p2p": 0.0036826663655330095,
      "iqr": 0.0
    },
    "Ch2": {
      "mean": 2.976725564805675,
      "rms": 0.0017420482089137268,
      "p2p": 0.003728701464197126,
      "iqr": 0.003

: 